In [ ]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import torch
from torch import nn
from sklearn.feature_extraction.text import CountVectorizer
from transformers import BertTokenizer
from transformers import BertModel
from datasets import Dataset
from transformers import AutoTokenizer, AutoModel, Trainer, TrainingArguments, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from catalyst import dl
from catalyst import dl, utils
from ignite.engine import Engine, Events
from ignite.metrics import Accuracy, Loss, RunningAverage
from ignite.handlers import ModelCheckpoint, EarlyStopping
from ignite.contrib.handlers import ProgressBar
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss
from sklearn.metrics import f1_score
import warnings
warnings.simplefilter('ignore')
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import DistilBertTokenizer, DistilBertModel
import logging
from util import *
import torch.nn.functional as F
from pytorch_metric_learning import miners, losses
import catalyst.contrib as contrib
from datetime import datetime
import pprint
from ignite.engine import _prepare_batch
from ignite.engine import create_supervised_trainer

In [ ]:
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    )

In [ ]:
training_set = pd.read_json("training_set.json.gz", lines=True, orient="records")
testing_set = pd.read_json("testing_set.json.gz", lines=True, orient="records")

In [ ]:
subset = list(sorted(set(all_tiers_100)-set(["PersonalizedProduct"])))
nice_subset = [tier_translations[x] for x in subset]

In [ ]:
model_name = "/home/martin/IdeaProjects/phenetics/bertForPatents/"
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModel.from_pretrained(model_name, gradient_checkpointing=True)

In [ ]:
training_set['labels']=training_set[subset].astype(int).values.tolist()
testing_set['labels']=testing_set[subset].astype(int).values.tolist()

In [ ]:
# cpc_embeddings = np.fromfile("/home/martin/patentmark/cpc.node2vec.emb.32d.bin", dtype=np.float32).reshape((-1,32))

# import joblib
# cpc_labelizer = joblib.load('./node2id.joblib')
# cpc_lookup = {c: n for n, c in enumerate(cpc_labelizer.classes_)}

# @f.collecting
# def convert_cpc_codes(codes):
#     for code in codes:
#         if code in cpc_lookup:
#             yield cpc_lookup[code]
    
# def embed_cpc_codes(codes):
#     embedding = np.zeros(32)
#     converted = convert_cpc_codes(codes)
    
#     if not converted:
#         return embedding
    
#     for code_id in converted:
#         embedding = embedding + cpc_embeddings[code_id]
        
#     return embedding / len(converted)

# training_set['embedded_cpc'] = training_set.cpc_codes.apply(embed_cpc_codes)
# testing_set['embedded_cpc'] = testing_set.cpc_codes.apply(embed_cpc_codes)


In [ ]:
#cpc_coder = CountVectorizer(analyzer=cpc_split, min_df=5)

In [ ]:
# training_set['citing'] = training_set[['citations', 'cited_by']].apply(
#         lambda row: list(set(row['citations']+row['cited_by'])), axis=1)

In [ ]:
# testing_set['citing'] = testing_set[['citations', 'cited_by']].apply(
#         lambda row: list(set(row['citations']+row['cited_by'])), axis=1)

In [ ]:
# training_set['people'] = training_set[['assignees', 'inventors']].apply(lambda row: list(set(row['assignees']+row['inventors'])), axis=1)

In [ ]:
# testing_set['people'] = testing_set[['assignees', 'inventors']].apply(lambda row: list(set(row['assignees']+row['inventors'])), axis=1)

In [ ]:
# def format(t):
#     CORP_TYPES = set(
#         [
#             "INC",
#             "LLC" "CORP",
#             "KK",
#             "SA",
#             "SRL",
#             "LTD",
#             "NL",
#             "PTY",
#             "AG",
#             "GMBH",
#             "KG",
#             "OG",
#             "LIMITED",
#             "SARL",
#             "BM",
#             "PLC",
#             "LP",
#             "IP",
#             "DBA",
#             "CORP",
#             "CO",
#         ]
#     )

#     tokenized = strip_non_alphanum(strip_punctuation(t)).upper().split(" ")
#     cleaned = [t for t in tokenized if t not in CORP_TYPES]
#     return "".join(cleaned)


# people_coder = CountVectorizer(analyzer=lambda x: map(format, x), min_df=2)

In [ ]:
# citing_coder = CountVectorizer(analyzer=lambda x: x, min_df=4)

In [ ]:
# citing_coder.fit(training_set.citing)
# len(citing_coder.vocabulary_)

In [ ]:
# people_coder.fit(training_set.people)

In [ ]:
# len(people_coder.vocabulary_)

In [ ]:
# #training_set['cpc_vec'] = list(cpc_coder.transform(training_set.cpc_codes).todense())
# #testing_set['cpc_vec'] = list(cpc_coder.transform(testing_set.cpc_codes).todense())
# training_set['people_vec'] = list(np.array(people_coder.transform(training_set.people).todense()))
# testing_set['people_vec'] = list(np.array(people_coder.transform(testing_set.people).todense()))
# training_set['citing_vec'] = list(np.array(citing_coder.transform(training_set.citing).todense()))
# testing_set['citing_vec'] = list(np.array(citing_coder.transform(testing_set.citing).todense()))

In [ ]:
# training_set = training_set.explode('labels').reset_index()
# testing_set = testing_set.explode('labels').reset_index()

In [ ]:
#training_set.labels.str.len().describe()

In [ ]:
#possible_labels = set([tier_translations[x] for x in subset])

In [ ]:
def tokenize(tokenizer, text, max_len):
        text = str(text)
        text = " ".join(text.split())

        inputs = tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        return {
            f"input_ids": torch.tensor(ids, dtype=torch.long),
            f"attention_mask": torch.tensor(mask, dtype=torch.long),
            f"token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
        }
    
def tokenize_list(tokenizer, text_list, max_len):
        
        inputs = tokenizer.batch_encode_plus(
            text_list,
            #None,
            #add_special_tokens=True,
            max_length=max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        return {
            f"input_ids": torch.tensor(ids, dtype=torch.long),
            f"attention_mask": torch.tensor(mask, dtype=torch.long),
            f"token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
        }

In [ ]:
# subset_tokenized = tokenize_list(tokenizer, nice_subset, 56)

In [ ]:
# import fse
# import gensim.downloader as api
# glove = api.load("glove-wiki-gigaword-100")

In [ ]:
# from fse import IndexedList, SplitIndexedList
# from fse.models import uSIF
# s = SplitIndexedList(nice_subset)
# label_model = uSIF(glove, workers=32, lang_freq="en")
# label_model.train(s)

In [ ]:
#subset_embeddings = label_model.infer(s)

In [ ]:
#subset_embeddings.shape

In [ ]:
class MultiLabelDataset(Dataset):

    def __init__(self, dataframe, tokenizer):
        self.tokenizer = tokenizer
        self.data = dataframe
        
        self.claims = dataframe.claims
        self.abstracts = dataframe.abstract
        
        self.labels = dataframe.labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        
        abstract = tokenize(tokenizer, self.abstracts[index], max_len=MAX_LEN_ABSTRACT)
        claims = tokenize(tokenizer, self.claims[index], MAX_LEN_CLAIMS)
        
        labels = torch.tensor(np.array(self.labels[index]), dtype=torch.float)
        
        #people = torch.tensor(np.array(self.data.people_vec[index]), dtype=torch.float)
        #citing = torch.tensor(np.array(self.data.citing_vec[index]), dtype=torch.float)
        #embedded_cpc = torch.tensor(np.array(self.data.embedded_cpc[index]), dtype=torch.float)        
                
        return {"abstract": abstract, 
                "claims": claims, 
                
                #'cpcs': cpcs,
                # 'people': people,
                # 'citing': citing,
                # 'embedded_cpc': embedded_cpc,
                 'labels': labels}

In [ ]:
training_dataset = MultiLabelDataset(training_set, tokenizer)
testing_dataset = MultiLabelDataset(testing_set, tokenizer)

In [ ]:
MAX_LEN_CLAIMS = 512
MAX_LEN_ABSTRACT = 160
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4
EPOCHS = 100
LEARNING_RATE = 1e-5
SEED = 17
PRED_THRES = 0.4
ACCUM_STEPS = 8
NUM_LABELS = len(nice_subset)

device = utils.get_device()
logdir="/var/patentmark/logdir/fit2/" + datetime.now().strftime("%Y%m%d-%H%M%S")

In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

training_loader = DataLoader(training_dataset, **train_params)
testing_loader = DataLoader(testing_dataset, **test_params)

In [ ]:
# class PatentModel(torch.nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.text_embedder = AutoModel.from_pretrained(model_name, gradient_checkpointing=True)
        
#         #self.people_embedder = torch.nn.Linear(len(people_coder.vocabulary_), 64)
#         #self.citing_embedder = torch.nn.Linear(len(citing_coder.vocabulary_), 64)
        
#         total_embedding_size = self.text_embedder.pooler.dense.out_features*2 #+32+64*2
#         output_size = 768 #self.text_embedder.pooler.dense.out_features
        
#         self.dropout1 = nn.Dropout(0.1)
        
#         self.dense1 = nn.Linear(total_embedding_size, output_size)
#         self.dense1label = nn.Linear(self.text_embedder.pooler.dense.out_features, output_size)
        
#         self.categorizer = nn.Linear(output_size, NUM_LABELS)

    
#     def encode_label(self, label):
#         label_emb = self.text_embedder(input_ids=label["input_ids"].to(device), attention_mask=label["attention_mask"].to(device))
#         label_emb = label_emb[0][:,0]
        
#         x = self.dropout1(label_emb)
#         x = F.elu(self.dense1label(x))
#         return x
    
#     def predict_classes(self, embeddings):
#         #x = self.dropout1(embeddings)
#         x = embeddings
#         x = self.categorizer(x)
#         return x
        
        
#     def encode_patent(self, abstract, claims): #, embedded_cpc, people, citing):
        
#         abstract_emb = self.text_embedder(input_ids=abstract["input_ids"].to(device), 
#                                           attention_mask=abstract["attention_mask"].to(device))
#         abstract_emb = abstract_emb[0][:, 0]
        
#         claim_emb = self.text_embedder(input_ids=claims["input_ids"].to(device), 
#                                        attention_mask=claims["attention_mask"].to(device))
#         claim_emb = claim_emb[0][:, 0]
        
# #         people_emb = F.elu(self.people_embedder(people.to(device)))
# #         citing_emb = F.elu(self.citing_embedder(citing.to(device)))
    
#         x = torch.cat((abstract_emb, claim_emb), 1)
#                        #, embedded_cpc.to(device), people_emb, citing_emb), 1)
#         x = self.dropout1(x)
#         x = F.elu(self.dense1(x))
        
#         return x

import torch.nn.functional as F

class BasicModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.text_embedder = AutoModel.from_pretrained(model_name, gradient_checkpointing=True)
        
        total_embedding_size = self.text_embedder.pooler.dense.out_features*2
        output_size = NUM_LABELS
        bottleneck = 768
    
        self.dropout1 = nn.Dropout(0.1)
        self.dense1 = nn.Linear(total_embedding_size, bottleneck)
        self.classifier = nn.Linear(bottleneck, output_size)

            
    def forward(self, abstract, claims):
        abstract_emb = self.text_embedder(input_ids=abstract["input_ids"], attention_mask=abstract["attention_mask"])
        abstract_emb = abstract_emb[0][:, 0]
        
        claim_emb = self.text_embedder(input_ids=claims["input_ids"], attention_mask=claims["attention_mask"])
        claim_emb = claim_emb[0][:, 0]
           
        x = torch.cat((abstract_emb, claim_emb), 1)
        x = self.dropout1(x)
        x = F.elu(self.dense1(x))
        x = self.classifier(x)
        
        return x


#model = PatentModel().to(device)
model = BasicModel().to(device)

In [ ]:
optimizer = torch.optim.AdamW(params =  model.parameters(), lr=LEARNING_RATE)
loss_function = torch.nn.BCEWithLogitsLoss()
#miner = miners.MultiSimilarityMiner()
#loss_function = losses.TripletMarginLoss()
#classifier_function = torch.nn.BCEWithLogitsLoss()

# from ignite.contrib.handlers.tensorboard_logger import *
# tb_logger = TensorboardLogger(log_dir=logdir)

In [ ]:
def basic_train_step(engine, batch):
    model.train()
    
    abstract = batch['abstract'].to(device)
    claims = batch['claims'].to(device)
    #people = batch['people']#.to(device)
    #citing = batch['citing']#.to(device)
    #embedded_cpc = batch['embedded_cpc']#.to(device)
    labels = batch['labels'].to(device)

    #label_embeddings = model.encode_label(subset_tokenized)
    
#     loss = 0
#     for label_idx in range(NUM_LABELS):
#         current_labels = labels[:, label_idx]
#         hard_pairs = miner(patent_emb, current_labels)
#         loss += loss_function(patent_emb, current_labels, hard_pairs)      

    patent_emb = model.encode_patent(abstract=abstract, claims=claims)
                                     #, embedded_cpc=embedded_cpc, people=people, citing=citing)    
    predictions = model.predict_classes(patent_emb)
    loss = classifier_function(predictions, labels)
    loss.backward()
    
    if engine.state.iteration % ACCUM_STEPS == 0:
        optimizer.step()
        optimizer.zero_grad()

    return loss.item()

trainer = Engine(train_step)


def validation_step(engine, batch):
    model.eval()
    abstract = batch['abstract']#.to(device)
    claims = batch['claims']#.to(device)
    #people = batch['people']#.to(device)
    #citing = batch['citing']#.to(device)
    #embedded_cpc = batch['embedded_cpc']#.to(device)
    labels = batch['labels'].to(device)
    
    with torch.no_grad():
        patent_emb = model.encode_patent(abstract=abstract, claims=claims)
                                         #, embedded_cpc=embedded_cpc, people=people, citing=citing)
        raw_predictions = model.predict_classes(patent_emb)
        return raw_predictions, labels

def thresholded_output_transform(output):
    y_pred, y = output
    y_pred = torch.round(torch.sigmoid(y_pred))
    return y_pred, y

#train_evaluator = Engine(validation_step)
test_evaluator = Engine(validation_step)

#Accuracy(is_multilabel=True, output_transform=thresholded_output_transform).attach(train_evaluator, 'train_accuracy')
#Loss(classifier_function).attach(train_evaluator, 'train_loss')
Accuracy(is_multilabel=True, output_transform=thresholded_output_transform).attach(test_evaluator, 'val_accuracy')
Loss(classifier_function).attach(test_evaluator, 'val_loss')

RunningAverage(output_transform=lambda x: x).attach(trainer, 'loss')
pbar = ProgressBar(persist=True, bar_format="")
pbar.attach(trainer, ['loss'])

@trainer.on(Events.ITERATION_COMPLETED(every=100))
def log_validation_results(engine):
    test_evaluator.run(testing_loader)
    metrics = test_evaluator.state.metrics
    pbar.log_message(
        "Val Results - Epoch: {} \nMetrics\n{}"
        .format(engine.state.epoch, pprint.pformat(metrics)))

trainer.run(training_loader, max_epochs=1)
    
# @trainer.on(Events.ITERATION_COMPLETED(every=log_interval))
# def log_training_loss(trainer):
#     print("Epoch[{}] Loss: {:.2f}".format(trainer.state.epoch, trainer.state.output))


# def log_validation_results(engine):
#     validation_evaluator.run(valid_iterator)
#     metrics = validation_evaluator.state.metrics
#     avg_accuracy = metrics['accuracy']
#     avg_bce = metrics['bce']
#     pbar.log_message(
#         "Validation Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.2f}"
#         .format(engine.state.epoch, avg_accuracy, avg_bce))
#     pbar.n = pbar.last_print_n = 0

In [ ]:


# log_interval = 10
# @trainer.on(Events.ITERATION_COMPLETED(every=log_interval))
# def log_validation_results(trainer):
#     print("running validation")
#     evaluator.run(testing_loader)
#     metrics = evaluator.state.metrics
#     print("Validation Results - Epoch: {}  Avg accuracy: {:.2f}"
#           .format(trainer.state.epoch, metrics["accuracy"]))
